In [1]:
from sklearn.datasets import load_wine
from sklearn.model_selection import train_test_split
import alibi
from alibi_detect.cd import TabularDrift

wine_data = load_wine()
feature_names = wine_data.feature_names
X, y = wine_data.data, wine_data.target
X_ref, X_test, y_ref, y_test = train_test_split(
    X, y,
    test_size=0.50,
    random_state=42
)

# Detectig drift

In [2]:
cd = TabularDrift(x_ref=X_ref, p_val=.01)
preds = cd.predict(X_test)
labels = ['No', 'Yes']
print('Drift: {}'.format(labels[preds['data']['is_drift']]))

Drift: No


In [3]:
X_test_cal_error = 1.1*X_test
preds = cd.predict(X_test_cal_error)
labels = ['No', 'Yes']
print('Drift: {}'.format(labels[preds['data']['is_drift']]))

Drift: Yes


# Hyperparameter tuning

In [5]:
from hyperopt import hp

space = {
    'warm_start' : hp.choice('warm_start', [True, False]),
    'fit_intercept' : hp.choice('fit_intercept', [True, False]),
    'tol' : hp.uniform('tol', 0.00001, 0.0001),
    'C' : hp.uniform('C', 0.05, 2.5),
    'solver' : hp.choice('solver', ['newton-cg', 'lbfgs',
    'liblinear']),
    'max_iter' : hp.choice('max_iter', range(10,500))
}

def objective(params, n_folds, X, y):
    # Perform n_fold cross validation with hyperparameters
    clf = LogisticRegression(**params, random_state=42)
    scores = cross_val_score(clf, X, y, cv=n_folds, scoring='f1_macro')
    # Extract the best score
    max_score = max(scores)
    # Loss must be minimized
    loss = 1 - max_score
    # Dictionary with information for evaluation
    return {
        'loss': loss,
        'params': params,
        'status': STATUS_OK
    }

# Auto sklearn



In [6]:
import numpy as np
import sklearn.datasets
import sklearn.metrics
import autosklearn.classification

X_train, X_test, y_train, y_test = train_test_split(
    X, y,
    test_size=0.50,
    random_state=42
)

automl = autosklearn.classification.AutoSklearnClassifier(
    time_left_for_this_task=60*3,
    per_run_time_limit=20,

)
automl.fit(X_train, y_train, X_test, y_test, dataset_name='wine')

[ERROR] [2022-05-25 10:52:33,878:Client-AutoML(1):wine] Dummy prediction failed with run state StatusType.CRASHED and additional output: {'error': 'Result queue is empty', 'exit_status': "<class 'pynisher.limit_function_call.AnythingException'>", 'subprocess_stdout': '', 'subprocess_stderr': 'Process pynisher function call:\nTraceback (most recent call last):\n  File "/Users/moberleitner/.pyenv/versions/3.9.10/lib/python3.9/multiprocessing/process.py", line 315, in _bootstrap\n    self.run()\n  File "/Users/moberleitner/.pyenv/versions/3.9.10/lib/python3.9/multiprocessing/process.py", line 108, in run\n    self._target(*self._args, **self._kwargs)\n  File "/Users/moberleitner/.pyenv/versions/3.9.10/envs/3.9.10-tf/lib/python3.9/site-packages/pynisher/limit_function_call.py", line 108, in subprocess_func\n    resource.setrlimit(resource.RLIMIT_AS, (mem_in_b, mem_in_b))\nValueError: current limit exceeds maximum limit\n', 'exitcode': 1, 'configuration_origin': 'DUMMY'}.


ValueError: Dummy prediction failed with run state StatusType.CRASHED and additional output: {'error': 'Result queue is empty', 'exit_status': "<class 'pynisher.limit_function_call.AnythingException'>", 'subprocess_stdout': '', 'subprocess_stderr': 'Process pynisher function call:\nTraceback (most recent call last):\n  File "/Users/moberleitner/.pyenv/versions/3.9.10/lib/python3.9/multiprocessing/process.py", line 315, in _bootstrap\n    self.run()\n  File "/Users/moberleitner/.pyenv/versions/3.9.10/lib/python3.9/multiprocessing/process.py", line 108, in run\n    self._target(*self._args, **self._kwargs)\n  File "/Users/moberleitner/.pyenv/versions/3.9.10/envs/3.9.10-tf/lib/python3.9/site-packages/pynisher/limit_function_call.py", line 108, in subprocess_func\n    resource.setrlimit(resource.RLIMIT_AS, (mem_in_b, mem_in_b))\nValueError: current limit exceeds maximum limit\n', 'exitcode': 1, 'configuration_origin': 'DUMMY'}.

# Autokeras

In [8]:
import autokeras as ak

clf = ak.StructuredDataClassifier(max_trials=5)
clf.fit(x=X_train, y=y_train, validation_data=(X_test, y_test))
accuracy=clf.evaluate(x=X_test, y=y_test)

INFO:tensorflow:Reloading Oracle from existing project ./structured_data_classifier/oracle.json
INFO:tensorflow:Reloading Tuner from ./structured_data_classifier/tuner0.json
INFO:tensorflow:Oracle triggered exit
INFO:tensorflow:Assets written to: ./structured_data_classifier/best_model/assets
[WARNING] [2022-05-25 11:01:54,788:tensorflow] Detecting that an object or model or tf.train.Checkpoint is being deleted with unrestored values. See the following logs for the specific values in question. To silence these warnings, use `status.expect_partial()`. See https://www.tensorflow.org/api_docs/python/tf/train/Checkpoint#restorefor details about the status object returned by the restore function.
[WARNING] [2022-05-25 11:01:54,789:tensorflow] Value in checkpoint could not be found in the restored object: (root).optimizer.iter
[WARNING] [2022-05-25 11:01:54,790:tensorflow] Value in checkpoint could not be found in the restored object: (root).optimizer.beta_1
[WARNING] [2022-05-25 11:01:54,79